# Unit 6: People Perception Part 3. People Tracking

In this final Unit about person HRI, you will learn how to make a robot follow people around and detect when they approach so that you can avoid them or stop. This skill is absolutely essential for robots in public spaces, like museums and malls. It's also important in HRI because a robot has to be able to follow a person, like its instructor, to go wherever it's needed.

<img src="img/perception_unit6_start.png"/>

To learn this, you will use a simulation with the Fetch robot in it and a person that you can move around. Try moving the person right now with the command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch person_sim move_person_standing.launch

Try moving the person around. Once you have finished, stop the launch file and continue with the Unit. The basic keys in order to controlthe person are the following:

<table style="width:100%">
  
  <tr>
    <th>
    <figure>
        <img src="img/key_i.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Move forward</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_comma.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Move backward</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_j.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Turn left</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_l.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Turn right</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_k.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Stop</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_q.png"width="40"></img>
        <img src="img/key_z.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Increase / Decrease Speed</p>
    </th> 
  </tr>
  
</table>

## ROS package for tracking people

There are various ways in which a person can be tracked:<br>

* **Detecting legs**: This is the most basic way of detecting people. You just look for laser patterns that have a U shape. As you might guess, this gives a lot of false positives, especially when there are chairs around because they get easily confused by the legs.
* **Detecting upper body**: This method detects people looking for patterns that match upper body shapes. More robust than leg detection.
* **Detecting pedestrians**: This is a merging of some data from ground_hog, visual_odometry, und upper_body_detector.

But, in the end, none of them are as strong as all of them combined. So the best algorithms are the ones that combine all of the data from these detecting systems and many others, and process them to have coherent detection.

You are going to learn how to launch each of these systems and, afterwards, combine them into only one system that publishes the final detection.

Before starting, as always, create a new package. In this case it will be called **my_people_tracker_pkg**:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd;cd ../src
catkin_create_pkg my_people_tracker_pkg geometry_msgs people_velocity_tracker rospy

## Leg Detector

To start the leg detector, you will use the <a href="http://wiki.ros.org/leg_detector">leg_detector</a> package. As input, it will take the **/base_scan** topic, which contains the laser readings' data.

<p style="background:green;color:white;">**leg_detector_start.launch**</p>

In [ ]:
<launch> 
    <arg name="scan" default="/base_scan" />
    <arg name="machine" default="localhost" />
    <arg name="user" default="" />
    <!-- Leg Detector -->
    <node pkg="leg_detector" type="leg_detector" name="leg_detector" args="scan:=$(arg scan) $(find leg_detector)/config/trained_leg_detector.yaml" respawn="true" output="screen">
        <param name="fixed_frame" type="string" value="odom" />
    </node>
    
    <!-- To PoseArray -->
    <include file="$(find detector_msg_to_pose_array)/launch/to_pose_array.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
    </include>
</launch>   

<p style="background:green;color:white;">**END leg_detector_start.launch**</p>

So, let's look at each element here:

In [ ]:
<arg name="scan" default="/base_scan" />
<arg name="machine" default="localhost" />
<arg name="user" default="" />

Here you are really just setting the machine and user to the default values. This is for ssh connections, but nothing that is relevant now.<br>
What is relevant is the **scan** topic that, in this case, is **/base_scan**. This topic is where the laser readings are published. These readings are the ones used by the **leg_detector**.

In [ ]:
<node pkg="leg_detector" type="leg_detector" name="leg_detector" args="scan:=$(arg scan) $(find leg_detector)/config/trained_leg_detector.yaml" respawn="true" output="screen">
    <param name="fixed_frame" type="string" value="odom" />
</node>

Here you are launching the **leg_detector**, taking the odom frame as the fixed_frame. But, the most important element here is the **trained_leg_detector.yaml** file. This is where all of the configuration parameters for the recognition are set.

In [ ]:
<!-- To PoseArray -->
<include file="$(find detector_msg_to_pose_array)/launch/to_pose_array.launch">
    <arg name="machine" value="$(arg machine)"/>
    <arg name="user" value="$(arg user)"/>
</include>

This last launch is quite strange. Why do you need to launch this **to_pose_array.launch** file? To understand why, you should first launch it, and then take a look at the topics published.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch my_people_tracker_pkg leg_detector_start.launch

Now take a look at the topics:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rostopic info /leg_tracker_measurements

In [ ]:
user ~ $ rostopic info /leg_tracker_measurements                                                                   Type: people_msgs/PositionMeasurementArray                                                                         Publishers:                                                                                                          * /leg_detector (http://ip-172-31-45-52:38842/)                                                                   Subscribers: None

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rostopic info /to_pose_array/leg_detector

In [ ]:
user ~ $ rostopic info /to_pose_array/leg_detector                                                                 Type: geometry_msgs/PoseArray                                                                                       Publishers:                                                                                                         * /to_pose_array (http://ip-172-31-45-52:50317/)                                                                   Subscribers: None

As you can see, these two topics publish position data. And the topic **/to_pose_array/leg_detector** is the one published by the **to_pose_array.launch** file. So, why do you need it? Basically, what this is doing is to transform the **/leg_tracker_measurements** data into **PoseArray**. This will be needed in the end, when you combine all detecting systems together. The only requirement for combining detectors is that all of them have to be in **PoseArray** format. So... because the leg detector doesn't generate that on its own, you have to tranform it.

Also, at the end of the pipeline, there is a topic called **/people_tracker_measurements**. This is where the final data will be published through all the people tracker systems.

Let's now <i>echo</i> this last topic mentioned:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rostopic echo /people_tracker_measurements

Now, try moving the person closer to the Fetch robot.<br>
As you can see, the detections are rare and don't always happen. You can, of course, change **trained_leg_detector.yaml** to try to improve the detections.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd leg_detector
cd config/
vim trained_leg_detector.yaml

But, as you can see, it's just a trivial matter.

<p style="background:#EE9023;color:white;">**Exercise U6-1**</p>

Create a script that takes the data from the leg detector system and makes the Fetch robot try to follow the person while it moves. See how it performs.

<p style="background:#EE9023;color:white;">**END Exercise U6-1**</p>

## Detect Upper Body

This system detects the upper part of a human's body and publishes its estimated location and orientation. It basically publishes a **PoseStamped** with position and orientation. It additionally publishes the RGB image with a square overlapping where the person is detected.

Here you have the file to launch it:

<p style="background:green;color:white;">**upper_body_start.launch**</p>

In [ ]:
<launch> 
    
    <arg name="machine" default="localhost" />
    <arg name="user" default="" />
    <arg name="load_params_from_file" default="true" />
    
    <arg name="ubd_queue_size" default="5" />
    <arg name="gp_queue_size" value="5" />
    
    <arg name="camera_namespace" value="/head_camera" />
    <arg name="rgb_image" value="/rgb/image_raw" />
    <arg name="depth_image" value="/depth_registered/image_raw" />
    <arg name="mono_image" value="/depth_registered/image_raw" />
    <arg name="camera_info_rgb" value="/rgb/camera_info" />
    <arg name="camera_info_depth" value="/depth_registered/camera_info" />
    
    <arg name="upper_body_detections" default="/upper_body_detector/detections" />
    <arg name="upper_body_bb_centres" default="/upper_body_detector/bounding_box_centres" />
    <arg name="upper_body_markers" default="/upper_body_detector/marker_array" />
    <arg name="upper_body_image" default="/upper_body_detector/image" />
    
    <arg name="ground_plane" default="/ground_plane" />
    
    
    <!-- Ground Plane -->
    <include file="$(find ground_plane_estimation)/launch/ground_plane_estimated.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="load_params_from_file" value="$(arg load_params_from_file)"/>
        <arg name="queue_size" value="$(arg gp_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="depth_image" value="$(arg depth_image)"/>
        <arg name="camera_info_rgb" value="$(arg camera_info_rgb)"/>
        <arg name="ground_plane" value="$(arg ground_plane)"/>
    </include>
    
    <!-- Upper Body Detector -->
    <include file="$(find upper_body_detector)/launch/upper_body_detector.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="load_params_from_file" value="$(arg load_params_from_file)"/>
        <arg name="queue_size" value="$(arg ubd_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="rgb_image" value="$(arg rgb_image)"/>
        <arg name="depth_image" value="$(arg depth_image)"/>
        <arg name="camera_info_depth" value="$(arg camera_info_depth)"/>
        <arg name="upper_body_detections" value="$(arg upper_body_detections)"/>
        <arg name="upper_body_bb_centres" value="$(arg upper_body_bb_centres)"/>
        <arg name="upper_body_markers" value="$(arg upper_body_markers)"/>
        <arg name="upper_body_image" value="$(arg upper_body_image)"/>
        <arg name="ground_plane" value="$(arg ground_plane)"/>
    </include>
</launch> 

<p style="background:green;color:white;">**END upper_body_start.launch**</p>

As you can see, this gets a bit more messy. But, essentially, you have to launch two things:<br>

* Launch the **upper_body_detector.launch**: This is the one that makes the recognition.
* Launch the **ground_plane_estimated.launch**: This is totally necessary for the upper_body_detector.launch to work, because it's subscribed to the /ground_plane topic. Knowing where the ground is allows it to calculate and detect the distance to where the upper body should be and look around there.

The topics for the image are pointed out:

In [ ]:
<arg name="camera_namespace" value="/head_camera" />
<arg name="rgb_image" value="/rgb/image_raw" />
<arg name="depth_image" value="/depth_registered/image_raw" />
<arg name="mono_image" value="/depth_registered/image_raw" />
<arg name="camera_info_rgb" value="/rgb/camera_info" />
<arg name="camera_info_depth" value="/depth_registered/camera_info" />

As you can see, they are divided into their main elements. If you want to use this with other robots, you will obviously have to change this part.<br>
So, when you launch **upper_body_start.launch**, you will get a bunch of topics related to the **upper_body** node, but these are the ones that are relevant for people tracking:

In [ ]:
/upper_body_detector/bounding_box_centres                                                                           /upper_body_detector/closest_bounding_box_centre                                                                   
/upper_body_detector/detections                                                                                     
/upper_body_detector/image                                                                                         
/upper_body_detector/marker_array

You have a **<i>bounding_box_center</i>** and a **<i>closest_bounding_box_center</i>** topics. These two are used to position the people detected in the space. The **closest** is used in case you want to follow the person that is closer to you from the start, for instance, and then follow them with other methods, in case of someone crossing between the robot and the person.

The **<i>detections</i>** topic provides the person detections that the system does, and the **<i>marker_array</i>** topic has this same data but in a marker format, which is very handy if you want a representation in RViz.

You can visualise the topic **/upper_body_detector/image**, selecting it in **rqt_imgae_view**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun rqt_image_view rqt_image_view

You should see something similar to this:

Back Upper body detection: 

<img src="img/perception_unit6_upperbody2.png"/>

Front upper body detection:

<img src="img/perception_unit6_upperbodyfront.png"/>

**NOTE**: As you can see, there is no effect on which way it's facing. Although it has problems when detecting a person sideways. So keep that in mind.<br>

<p style="background:#EE9023;color:white;">**Exercise U6-2**</p>

Create a new Python script that takes the data from the upper body topics and makes the Fetch robot try to follow the person while it moves. See how it performs.<br>
Compare it to the previous leg detector.

<p style="background:#EE9023;color:white;">**END Exercise U6-2**</p>

## Pedestrian detector:

This pedestrian detector uses the **detection of the ground** to filter false positives, and other algorithms to filter even more and make better predictions, including the **upper body** detections.

Here you have an example of how to use this detector with the Fetch robot:

<p style="background:green;color:white;">**mdl_pedestrian_start.launch**</p>

In [ ]:
<launch> 
    
    <arg name="machine" default="localhost" />
    <arg name="user" default="" />
    <arg name="load_params_from_file" default="true" />
    
    <arg name="gp_queue_size" value="5" />
    <arg name="pt_queue_size" value="10" />
    <arg name="vo_queue_size" value="5" />
    <arg name="ubd_queue_size" value="5" />
    
    <arg name="camera_namespace" value="/head_camera" />
    <arg name="rgb_image" value="/rgb/image_raw" />
    <arg name="depth_image" value="/depth_registered/image_raw" />
    <arg name="mono_image" value="/depth_registered/image_raw" />
    <arg name="camera_info_rgb" value="/rgb/camera_info" />
    <arg name="camera_info_depth" value="/depth_registered/camera_info" />
    
    <arg name="upper_body_detections" default="/upper_body_detector/detections" />
    <arg name="upper_body_bb_centres" default="/upper_body_detector/bounding_box_centres" />
    <arg name="upper_body_markers" default="/upper_body_detector/marker_array" />
    <arg name="upper_body_image" default="/upper_body_detector/image" />
    
    <arg name="ground_plane" default="/ground_plane" />
    
    <arg name="visual_odometry" default="/visual_odometry/motion_matrix" />
    
    <arg name="people_array" value="/mdl_people_tracker/people_array" />
    <arg name="people_image" value="/mdl_people_tracker/image" />
    <arg name="people_markers" value="/mdl_people_tracker/marker_array" />
    <arg name="people_poses" value="/mdl_people_tracker/pose_array" />
    
    <arg name="tf_target_frame" value="/base_link" />
    
    <!-- Visual Odometry -->
    <include file="$(find visual_odometry)/launch/visual_odometry.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="queue_size" value="$(arg vo_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="depth_image" value="$(arg depth_image)"/>
        <arg name="mono_image" value="$(arg mono_image)"/>
        <arg name="camera_info_depth" value="$(arg camera_info_depth)"/>
        <arg name="motion_parameters" value="$(arg visual_odometry)"/>
    </include>
    
    <!-- Ground Plane -->
    <include file="$(find ground_plane_estimation)/launch/ground_plane_estimated.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="load_params_from_file" value="$(arg load_params_from_file)"/>
        <arg name="queue_size" value="$(arg gp_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="depth_image" value="$(arg depth_image)"/>
        <arg name="camera_info_rgb" value="$(arg camera_info_rgb)"/>
        <arg name="ground_plane" value="$(arg ground_plane)"/>
    </include>
    
    <!-- Upper Body Detector -->
    <include file="$(find upper_body_detector)/launch/upper_body_detector.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="load_params_from_file" value="$(arg load_params_from_file)"/>
        <arg name="queue_size" value="$(arg ubd_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="rgb_image" value="$(arg rgb_image)"/>
        <arg name="depth_image" value="$(arg depth_image)"/>
        <arg name="camera_info_depth" value="$(arg camera_info_depth)"/>
        <arg name="upper_body_detections" value="$(arg upper_body_detections)"/>
        <arg name="upper_body_bb_centres" value="$(arg upper_body_bb_centres)"/>
        <arg name="upper_body_markers" value="$(arg upper_body_markers)"/>
        <arg name="upper_body_image" value="$(arg upper_body_image)"/>
        <arg name="ground_plane" value="$(arg ground_plane)"/>
    </include>
    
    <!-- Pedestrian Tracking -->
    <include file="$(find mdl_people_tracker)/launch/mdl_people_tracker.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="load_params_from_file" value="$(arg load_params_from_file)"/>
        <arg name="queue_size" value="$(arg pt_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="rgb_image" value="$(arg rgb_image)"/>
        <arg name="camera_info_rgb" value="$(arg camera_info_rgb)"/>
        <arg name="ground_plane" value="$(arg ground_plane)"/>
        <arg name="upper_body_detections" value="$(arg upper_body_detections)"/>
        <arg name="visual_odometry" value="$(arg visual_odometry)"/>
        <arg name="people_array" value="$(arg people_array)"/>
        <arg name="people_image" value="$(arg people_image)"/>
        <arg name="people_markers" value="$(arg people_markers)"/>
        <arg name="people_poses" value="$(arg people_poses)"/>
        <arg name="target_frame" value="$(arg tf_target_frame)"/>
    </include>
</launch> 

<p style="background:green;color:white;">**END mdl_pedestrian_start.launch**</p>

Although it might seem enormous, you can see that everything, except a few things related to the **mdl_people_tracker.launch,** is exactly the same as what you already know. That's because this mdl works on the previous data of UpperBody and ground, plus using visual_odometry.

When you execute this, you will get a few topics. These are the most relevant:

In [ ]:
/mdl_people_tracker/image                                                                                           /mdl_people_tracker/marker_array                                                                                   
/mdl_people_tracker/people_array                                                                                   
/mdl_people_tracker/pose_array

As before, you have some topics publishing Poses, and other topics publishing markers which contain this Pose information as markers in order to be able to visualize it in RVIZ.<br>

You can also visualize the topic **/mdl_people_tracker/image**, in this case, with a cylinder around the detection.

Back pedestrian detector:

<img src="img/perception_unit6_mdl1.png"/>

Front pedestrian detector:

<img src="img/perception_unit6_mdlfront.png"/>

**NOTE**: Observe that if it loses the person, the cylinder will probably change its color, considering it another person. So keep that in mind.

<p style="background:#EE9023;color:white;">**Exercise U6-3**</p>

Create a new script that takes the data from the mld (pedestrian detector) topics and makes the Fetch robot try to follow the person while it moves. See how it performs.

Compare it to the previous one.

<p style="background:#EE9023;color:white;">**END Exercise U6-3**</p>

## Combining it all together

And, finally, once you have all of these systems working, you are now going to use all of the information and process it with the package **spencer_people_tracking**.<br>
To launch it, one of the most important things is the **configuration file**, in which you add all of the so-called **detectors,** plus the detection by speed model.<br>
You unite all of this information through a <a href="https://en.wikipedia.org/wiki/Kalman_filter">Kalman filter</a>, more precisely a <a href="https://www.seas.harvard.edu/courses/cs281/papers/unscented.pdf">Unscentered Kalman Filter</a>. Kalman filters are used to extract conclusions from sets of data that, by themselves, are inaccurate, but combined with other data sources, are more precise.<br>

Here you have an example of the config file:

<p style="background:green;color:white;">**detectors.yaml**</p>

In [ ]:
bayes_people_tracker:
    filter_type: "UKF"                                         # The Kalman filter type: EKF = Extended Kalman Filter, UKF = Uncented Kalman Filter
    cv_noise_params:                                           # The noise for the constant velocity prediction model
        x: 1.4
        y: 1.4
    detectors:                                                 # Add detectors under this namespace
        upper_body_detector:                                   # Name of detector (used internally to identify them). Has to be unique.
            topic: "/upper_body_detector/bounding_box_centres" # The topic on which the geometry_msgs/PoseArray is published
            cartesian_noise_params:                            # The noise for the cartesian observation model
                x: 0.5
                y: 0.5
            matching_algorithm: "NNJPDA"                       # The algorthim to match different detections. NN = Nearest Neighbour, NNJPDA = NN Joint Probability Data Association
        leg_detector:                                          # Name of detector (used internally to identify them). Has to be unique.
            topic: "/to_pose_array/leg_detector"               # The topic on which the geometry_msgs/PoseArray is published
            cartesian_noise_params:                            # The noise for the cartesian observation model
                x: 0.2
                y: 0.2
            matching_algorithm: "NNJPDA"                       # The algorthim to match different detections. NN = Nearest Neighbour, NNJPDA = NN Joint Probability Data Association
        mobility_aid_detector:                                 # Name of detector (used internally to identify them). Has to be unique.
            topic: "/mobility_aid_detections"                  # The topic on which the geometry_msgs/PoseArray is published
            cartesian_noise_params:                            # The noise for the cartesian observation model
                x: 0.25
                y: 0.25
            matching_algorithm: "NNJPDA"                       # The algorthim to match different detections. NN = Nearest Neighbour, NNJPDA = NN Joint Probability Data Association
        mdl_people_detector:                                 # Name of detector (used internally to identify them). Has to be unique.
            topic: "/mdl_people_tracker/pose_array"                  # The topic on which the geometry_msgs/PoseArray is published
            cartesian_noise_params:                            # The noise for the cartesian observation model
                x: 0.2
                y: 0.2
            matching_algorithm: "NNJPDA" 

The most important value here is the **<i>cartesian_noise_params</i>**. This will regulate the impact in the final decision that the detector makes. And which values do you have to state here?<br>
Well, that's a good question. There are two methods that you can follow:<br>

* **Extract the statistical values of each detector system**: This method consists of executing only that detector system in a controlled environment. You, then, save all the X and Y values of the pose array. The more you have, the better the statistical values that you will get. Once done, generate the mean and <a href="https://en.wikipedia.org/wiki/Standard_deviation">standard deviation</a> of the X and Y separately. You will end up with a Standard Deviation for X and a Standard Deviation for Y. **These values are the ones you will have to use as <i>cartesian_noise_params</i>**.<br>

<img src="img/perception_unit6_standard_deviation.png"/>

<p>By <a href="//commons.wikimedia.org/wiki/User:Mwtoews" title="User:Mwtoews">M. W. Toews</a> - <span class="int-own-work" lang="en">Own work</span>, based (in concept) on figure by Jeremy Kemp, on 2005-02-09, <a href="http://creativecommons.org/licenses/by/2.5" title="Creative Commons Attribution 2.5">CC BY 2.5</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=1903871">Link</a>**</p>

* Start with the example values provided and change them until you get the best results. This way of working, although not at all scientific, its very common in engineering testing. Sometimes, you are much faster with this method, just iterating over values and selecting the best results, rather than extracting all the theoretical data or models. This is due to the fact that you may not have all the real elements in mind when you create your theoretical tests.

If you want to know more details about the implementation of this system, please refer to the <a href="http://eprints.lincoln.ac.uk/17545/1/dondrup.pdf">original paper</a>  from the <a href="http://staff.lincoln.ac.uk">University of Lincoln</a>.

<p style="background:green;color:white;">**END detectors.yaml**</p>

The launch will be divided into two launches for more flexibility in changing arguments. This first one, **people_tracker_custom.launch,** is the core. Here, all the systems are launched. This includes: visual_odometry, ground_plane_estimation, upper_body_detector, pedestrian tracker (mdl_people_tracker), leg_detector, people_tracker, and logging.

<p style="background:green;color:white;">**people_tracker_custom.launch**</p>

In [ ]:
<launch>
    <!-- Global paramters -->
    <arg name="load_params_from_file" default="true" />
    <arg name="gp_queue_size" default="5" />
    <arg name="vo_queue_size" default="5" />
    <arg name="ubd_queue_size" default="5" />
    <arg name="pt_queue_size" default="10" />
    <arg name="camera_namespace" default="/camera" />
    <arg name="rgb_image" default="/rgb/image_rect_color" />
    <arg name="depth_image" default="/depth/image_rect" />
    <arg name="mono_image" default="/rgb/image_mono" />
    <arg name="camera_info_rgb" default="/rgb/camera_info" />
    <arg name="camera_info_depth" default="/depth/camera_info" />
    <arg name="ground_plane" default="/ground_plane" />
    <arg name="upper_body_detections" default="/upper_body_detector/detections" />
    <arg name="upper_body_bb_centres" default="/upper_body_detector/bounding_box_centres" />
    <arg name="upper_body_markers" default="/upper_body_detector/marker_array" />
    <arg name="upper_body_image" default="/upper_body_detector/image" />
    <arg name="visual_odometry" default="/visual_odometry/motion_matrix" />
    <arg name="people_array" default="/mdl_people_tracker/people_array" />
    <arg name="people_image" default="/mdl_people_tracker/image" />
    <arg name="people_markers" default="/mdl_people_tracker/marker_array" />
    <arg name="people_poses" default="/mdl_people_tracker/pose_array" />
    <arg name="tf_target_frame" default="/base_link" />
    
    <arg name="scan" default="/base_scan" />

    <arg name="machine" default="localhost" />
    <arg name="user" default="" />
    
    <arg name="log" default="false" />
    <arg name="manager_topic" default="" />
    
    <arg name="tracker_filter_positions" default="/people_tracker_filter/positions" />
    
    <arg name="tracker_filter_pose" default="/people_tracker_filter/pose" />
    <arg name="tracker_filter_pose_array" default="/people_tracker_filter/pose_array" />
    <arg name="tracker_filter_people" default="/people_tracker_filter/people" />
    <arg name="tracker_filter_marker" default="/people_tracker_filter/marker_array" />
    
    <arg name="bayes_people_param_file" default="$(find bayes_people_tracker)/config/detectors.yaml" />
    <arg name="bayes_people_positions" default="/people_tracker/positions" />
    <arg name="bayes_people_pose" default="/people_tracker/pose" />
    <arg name="bayes_people_pose_array" default="/people_tracker/pose_array" />
    <arg name="bayes_people_people" default="/people_tracker/people" />
    <arg name="bayes_people_marker" default="/people_tracker/marker_array" />

    <machine name="$(arg machine)" address="$(arg machine)" env-loader="$(optenv ROS_ENV_LOADER )" user="$(arg user)" default="true"/>

    <!-- Visual Odometry -->
    <include file="$(find visual_odometry)/launch/visual_odometry.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="queue_size" value="$(arg vo_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="depth_image" value="$(arg depth_image)"/>
        <arg name="mono_image" value="$(arg mono_image)"/>
        <arg name="camera_info_depth" value="$(arg camera_info_depth)"/>
        <arg name="motion_parameters" value="$(arg visual_odometry)"/>
    </include>

    <!-- Ground Plane -->
    <include file="$(find ground_plane_estimation)/launch/ground_plane_estimated.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="load_params_from_file" value="$(arg load_params_from_file)"/>
        <arg name="queue_size" value="$(arg gp_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="depth_image" value="$(arg depth_image)"/>
        <arg name="camera_info_rgb" value="$(arg camera_info_rgb)"/>
        <arg name="ground_plane" value="$(arg ground_plane)"/>
    </include>

    <!-- Upper Body Detector -->
    <include file="$(find upper_body_detector)/launch/upper_body_detector.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="load_params_from_file" value="$(arg load_params_from_file)"/>
        <arg name="queue_size" value="$(arg ubd_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="rgb_image" value="$(arg rgb_image)"/>
        <arg name="depth_image" value="$(arg depth_image)"/>
        <arg name="camera_info_depth" value="$(arg camera_info_depth)"/>
        <arg name="upper_body_detections" value="$(arg upper_body_detections)"/>
        <arg name="upper_body_bb_centres" value="$(arg upper_body_bb_centres)"/>
        <arg name="upper_body_markers" value="$(arg upper_body_markers)"/>
        <arg name="upper_body_image" value="$(arg upper_body_image)"/>
        <arg name="ground_plane" value="$(arg ground_plane)"/>
    </include>

    <!-- Pedestrian Tracking -->
    <include file="$(find mdl_people_tracker)/launch/mdl_people_tracker.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="load_params_from_file" value="$(arg load_params_from_file)"/>
        <arg name="queue_size" value="$(arg pt_queue_size)"/>
        <arg name="camera_namespace" value="$(arg camera_namespace)"/>
        <arg name="rgb_image" value="$(arg rgb_image)"/>
        <arg name="camera_info_rgb" value="$(arg camera_info_rgb)"/>
        <arg name="ground_plane" value="$(arg ground_plane)"/>
        <arg name="upper_body_detections" value="$(arg upper_body_detections)"/>
        <arg name="visual_odometry" value="$(arg visual_odometry)"/>
        <arg name="people_array" value="$(arg people_array)"/>
        <arg name="people_image" value="$(arg people_image)"/>
        <arg name="people_markers" value="$(arg people_markers)"/>
        <arg name="people_poses" value="$(arg people_poses)"/>
        <arg name="target_frame" value="$(arg tf_target_frame)"/>
    </include>
    
    <!-- Leg Detector -->
    <node pkg="leg_detector" type="leg_detector" name="leg_detector" args="scan:=$(arg scan) $(find leg_detector)/config/trained_leg_detector.yaml" respawn="true" output="screen">
        <param name="fixed_frame" type="string" value="odom" />
    </node>
    
    
    <!-- People Tracker -->
    <include file="$(find bayes_people_tracker)/launch/people_tracker.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="param_file" value="$(arg bayes_people_param_file)"/>
        <arg name="target_frame" value="$(arg tf_target_frame)"/>
        <arg name="positions" value="$(arg bayes_people_positions)"/>
        <arg name="pose" value="$(arg bayes_people_pose)"/>
        <arg name="pose_array" value="$(arg bayes_people_pose_array)"/>
        <arg name="people" value="$(arg bayes_people_people)"/>
        <arg name="marker" value="$(arg bayes_people_marker)"/>
    </include>
    
    <!-- To PoseArray -->
    <include file="$(find detector_msg_to_pose_array)/launch/to_pose_array.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
    </include>
    
    
   <!-- Logging -->
    <include file="$(find bayes_people_tracker_logging)/launch/logging.launch">
        <arg name="machine" value="$(arg machine)"/>
        <arg name="user" value="$(arg user)"/>
        <arg name="log" value="$(arg log)"/>
        <arg name="manager_topic" value="$(arg manager_topic)"/>
        <arg name="positions" value="$(arg tracker_filter_positions)"/>
    </include>
</launch> 


<p style="background:green;color:white;">**END people_tracker_custom.launch**</p>

This second launch only remaps the values to your particular case.

<p style="background:green;color:white;">**start_peopletracker_custom.launch**</p>

In [ ]:
<launch>
    <!-- Start navigation -->
    <!-- rostopic info /people_tracker/marker_array -->
    <include file="$(find my_people_tracker_pkg)/launch/people_tracker_custom.launch" >
        <arg name="load_params_from_file" value="true" />
        
        <arg name="gp_queue_size" value="5" />
        <arg name="vo_queue_size" value="5" />
        <arg name="ubd_queue_size" value="5" />
        <arg name="pt_queue_size" value="10" />
        
        <arg name="camera_namespace" value="/head_camera" />
        <arg name="rgb_image" value="/rgb/image_raw" />
        <arg name="depth_image" value="/depth_registered/image_raw" />
        <arg name="mono_image" value="/depth_registered/image_raw" />
        <arg name="camera_info_rgb" value="/rgb/camera_info" />
        <arg name="camera_info_depth" value="/depth_registered/camera_info" />
        
        <arg name="ground_plane" value="/ground_plane" />
        <arg name="upper_body_detections" value="/upper_body_detector/detections" />
        <arg name="upper_body_bb_centres" value="/upper_body_detector/bounding_box_centres" />
        <arg name="upper_body_markers" value="/upper_body_detector/marker_array" />
        <arg name="upper_body_image" value="/upper_body_detector/image" />
        <arg name="visual_odometry" value="/visual_odometry/motion_matrix" />
        <arg name="people_array" value="/mdl_people_tracker/people_array" />
        <arg name="people_image" value="/mdl_people_tracker/image" />
        <arg name="people_markers" value="/mdl_people_tracker/marker_array" />
        <arg name="people_poses" value="/mdl_people_tracker/pose_array" />
        
        <arg name="bayes_people_param_file" default="$(find my_people_tracker_pkg)/config/detectors.yaml" />
        
        <arg name="tf_target_frame" value="/base_link" />
    
        <arg name="machine" value="localhost" />
        <arg name="user" value="" />
    </include>
</launch>

<p style="background:green;color:white;">**END start_peopletracker_custom.launch**</p>

Once you launch the **start_peopletracker_custom.launch** file, you will have access to the following topics. Be aware that sometimes it maw take a few seconds for the topics to start publishing meaningfull data, so be patient:<br>

* **/people_tracker/marker_array**: Position and orientation in Marker Format
* Topics related to the poeple detected and positions:

In [ ]:
/people_tracker/people                                                                                         
/people_tracker/pose
/people_tracker/pose_array
/people_tracker/positions

And these markers have the final positions of the detections of the people. But, there is more. You can visualize these markers in RViz, so let's take a look. To add them, you have to add a **MarkerArray** element and select the marker topic.

<img src="img/perception_unit6_markersclosup.png" />

You can also add some extra elements, like the laser readings and the camera, to see the detection markers imposed and how precise they are. You can also add the image of the mlc, for example, to keep track.

You should get something similar to this:

MarkerArray in shape of person:

<img src="img/perception_unit6_peopletrackmarkers.png"/>

Laser and pedestrian detection added to RVIZ:

<img src="img/perception_unit6_peopletrackallrviz.png"/>

Now, move the person around and see how the system performs. It won't be the perfect system, but it gives you a good detection and it's relatively robust.

<img src="img/perception_unit6_peopletrackingdemo.gif"/>

<p style="background:#EE9023;color:white;">**Exercise U6-4**</p>

* Create a script that follows the person based on the position topic. Compare it with the rest of the scripts that you did before.

* As an extra, add a new MarkerArray element to the RVIZ visualization. Then, select the right topic for upper_body markers to appear.

<img src="img/perception_unit6_upperbodymarker.png"/>

<p style="background:#EE9023;color:white;">**END Exercise U6-4**</p>

More information here: https://github.com/spencer-project/spencer_people_tracking

<p style="background:#EE9023;color:white;">**EXTRA Exercise U6-5**</p>

Implement a new detection system and add it to the detection pile. Use what you have learned in this perception course, like facial recognition, color blob tracking, or even object recognition.<br>
You could, for instance, create a color tracking system to follow only a person that has a certain color t-shirt. If it's robust enough, change the **<i>cartesian_noise_params</i>** accordingly.

<p style="background:#EE9023;color:white;">**END EXTRA Exercise U6-5**</p>

## Congratulations! You have finished the main topics of this course. You can now proceed to the final project to apply all that you have learned.

<p style="background:#417FB1;color:white;">**Project**</p>

You can now do the first exercise of the Aibo Project. There, you will have to make the Aibo Robot follow the small boy, Timmy, who is moving around the world.

<p style="background:#417FB1;color:white;">**END Project**</p>